In [ ]:
import torchvision.models as models
import torchvision.transforms as transforms
from torch import nn
from torchvision.utils import save_image


In [ ]:
# Preprocessing
def image_to_tensor(image):
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 512)),  # Resize for faster processing
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0).to(device)

def tensor_to_image(tensor):
    tensor = tensor.clone().detach().cpu().squeeze(0)
    tensor = tensor * torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    tensor = tensor + torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    tensor = torch.clamp(tensor, 0, 1)
    return np.array(transforms.ToPILImage()(tensor))

# Content and Style Loss
class ContentLoss(nn.Module):
    def __init__(self, target):
        super().__init__()
        self.target = target.detach()

    def forward(self, x):
        self.loss = F.mse_loss(x, self.target)
        return x

class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super().__init__()
        self.target = self.gram_matrix(target_feature).detach()

    def gram_matrix(self, input):
        b, c, h, w = input.size()
        features = input.view(c, h * w)
        G = torch.mm(features, features.t())
        return G.div(c * h * w)

    def forward(self, x):
        G = self.gram_matrix(x)
        self.loss = F.mse_loss(G, self.target)
        return x


In [ ]:
def get_style_model_and_losses(cnn, style_img, content_img):
    cnn = cnn.to(device).eval()
    content_layers = ['conv_4']
    style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

    model = nn.Sequential()
    content_losses = []
    style_losses = []
    i = 0  # increment every time a conv is added

    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = f'conv_{i}'
        elif isinstance(layer, nn.ReLU):
            name = f'relu_{i}'
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = f'pool_{i}'
        elif isinstance(layer, nn.BatchNorm2d):
            name = f'bn_{i}'
        else:
            continue

        model.add_module(name, layer)

        if name in content_layers:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module(f"content_loss_{i}", content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module(f"style_loss_{i}", style_loss)
            style_losses.append(style_loss)

    # Trim the model after last loss layer
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break
    model = model[:i + 1]

    return model, style_losses, content_losses


In [ ]:
def run_style_transfer(cnn, content_img, style_img, num_steps=200, style_weight=1e6, content_weight=1):
    model, style_losses, content_losses = get_style_model_and_losses(cnn, style_img, content_img)
    input_img = content_img.clone()

    optimizer = torch.optim.LBFGS([input_img.requires_grad_()])

    run = [0]
    while run[0] <= num_steps:
        def closure():
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = sum(sl.loss for sl in style_losses)
            content_score = sum(cl.loss for cl in content_losses)

            loss = style_score * style_weight + content_score * content_weight
            loss.backward()

            run[0] += 1
            return loss

        optimizer.step(closure)

    input_img.data.clamp_(0, 1)
    return input_img


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn = models.vgg19(pretrained=True).features.to(device).eval()

results_nst = []

print("Applying Neural Style Transfer...")

for i in range(min(len(cityscapes_images), len(dark_zurich_images))):
    content_img = image_to_tensor(cityscapes_images[i])
    style_img = image_to_tensor(dark_zurich_images[i])
    output_nst_c2d = run_style_transfer(cnn, content_img, style_img)

    content_img = image_to_tensor(dark_zurich_images[i])
    style_img = image_to_tensor(cityscapes_images[i])
    output_nst_d2c = run_style_transfer(cnn, content_img, style_img)

    output_img_c2d = tensor_to_image(output_nst_c2d)
    output_img_d2c = tensor_to_image(output_nst_d2c)

    results_nst.append((cityscapes_images[i], dark_zurich_images[i], output_img_c2d, output_img_d2c))


In [ ]:
for i, (cityscapes_img, dark_img, nst_c2d, nst_d2c) in enumerate(results_nst):
    images = [cityscapes_img, dark_img, nst_c2d, nst_d2c]
    titles = ["Cityscapes", "Dark Zurich", "NST: Cityscapes→Dark", "NST: Dark→Cityscapes"]

    fig = visualize_images(images, titles, rows=2, cols=2, figsize=(14, 10))
    plt.savefig(f"nst_result_{i+1}.png")
    plt.show()
    plt.close(fig)

print("NST completed. Results saved.")
